Предобработка данных и их подготовка к анализу и подаче в модель.

Под предобработкой понимаются следующие этапы работы с данными:

очистка данных от аномальных значений (выбросов);
работа с пропущенными значениями;
удаление признаков, которые не несут полезной информации;
создание новых признаков;
преобразование признаков и приведение данных к необходимому для анализа и модели формату.
Этап подготовки данных является самым трудоёмким и времязатратным при работе с любой бизнес-задачей. В среднем он занимает около 60-70% общей работы специалиста!

Почему этот этап так важен?

Ответ на этот вопрос кроется в распространённой среди специалистов поговорке «Мусор на входе — мусор на выходе», что означает следующее: если данные плохо подготовлены, то и результат прогнозирования даже самой мощной в мире нейронной сети будет сильно разниться с действительностью.

FEATURE ENGINEERING

Одним из этапов подготовки данных является удаление, преобразование и создание столбцов таблицы.

Зачем нужны такие манипуляции? 

Оказывается, что при правильных преобразованиях таблицы можно добиваться лучшего качества прогноза, а также извлекать новую информацию из данных и интерпретировать её для заказчика. 

Такой подход часто называют Feature Engineering, или генерацией признаков (фичей).

→ На самом деле, в этом термине заложен более глубокий смысл, ведь Feature Engineering — это целая методология получения более качественных и более производительных моделей за счёт манипуляций над данными. Специалисты часто называют данную методологию настоящим искусством, которое может быть освоено лишь с годами практики решения задач, ведь необходимо быть экспертом в исследуемой предметной области, чтобы понимать, как признаки влияют друг на друга и какое преобразование стоит к ним применить. 


На протяжении всего модуля мы будем производить множество тренировочных преобразований с нашей таблицей. Поэтому, чтобы не переопределять переменную melb_data и тем самым не повредить первоначальный DataFrame, создадим копию melb_df с помощью метода copy():

melb_df = melb_data.copy()
melb_df.head()

Лайфхак. Старайтесь всегда оставлять переменную с первоначальным DataFrame неизменной, создавайте копию исходной таблицы и совершайте преобразования на ней. Это оградит вас от ошибок, которые можно совершить при подготовке данных. Например, если вы понимаете, что преобразование оказалось неудачным, достаточно будет лишь запустить ячейку, в которой вы производите копирование, а не читать таблицу заново. Особенно критичным это может быть, когда количество строк в таблице исчисляется миллионами и её чтение занимает до нескольких минут.

УДАЛЕНИЕ СТОЛБЦОВ

→ Среди списка базовых операций над столбцами в Pandas важное место занимает возможность удаления столбцов из таблицы. Это может быть полезно, например, когда в данных есть признаки, которые не несут полезной информации.

Представим, что мы хотим построить модель, которая бы предсказывала цену объекта недвижимости в Мельбурне. Даже не будучи профессиональными риелторами, мы можем легко сделать следующие выводы:

цена объекта никак не зависит от его порядкового номера (столбец index);
признак, описывающий долготу и широту в виде кортежа Coordinates, дублирует информацию, представленную в столбцах Longitude и Lattitude.
За удаление строк и столбцов в таблице отвечает метод drop().

Основные параметры метода drop()

labels — порядковые номера или имена столбцов, которые подлежат удалению; если их несколько, то передаётся список;
axis — ось совершения операции, axis=0 — удаляются строки, axis=1 — удаляются столбцы;
inplace — если параметр выставлен на True, происходит замена изначального DataFrame на новый, при этом метод ничего не возвращает; если на False — возвращается копия DataFrame, из которой удалены указанные строки (столбцы), при этом первоначальный DataFrame не изменяется; по умолчанию параметр равен False.

Удалим столбцы index и Coordinates из таблицы с помощью метода drop(). Выведем первые пять строк таблицы и убедимся, что всё прошло успешно.

melb_df = melb_df.drop(['index', 'Coordinates'], axis=1)
melb_df.head()
Альтернативный вариант:

melb_df.drop(['index','Coordinates'],axis=1,inplace=True)
melb_df.head()

МАТЕМАТИЧЕСКИЕ ОПЕРАЦИИ СО СТОЛБЦАМИ

→ Pandas поддерживает базовые математические операции между столбцами: столбцы можно складывать, вычитать, умножать, делить между собой, а также возводить в степень. С помощью таких операций мы можем создавать новые признаки или производить преобразования над старыми.

Причём все операции со столбцами совершаются поэлементно, очень быстро, а самое главное — без написания циклов.

Такая производительность достигается за счёт того, что все математические операции со столбцами выполняются на языке программирования С, что значительно повышает скорость вычислений по сравнению с перебором элементов в цикле. 

Например, давайте создадим переменную total_rooms, в которой будем хранить общее количество комнат в здании. Для этого выполним сложение столбцов с количеством комнат, ванн и спален:

total_rooms = melb_df['Rooms'] + melb_df['Bedroom'] + melb_df['Bathroom']
display(total_rooms)

А теперь введём признак MeanRoomsArea, который соответствует средней площади одной комнаты для каждого объекта. Для этого разделим площадь здания на полученное ранее общее количество комнат:

melb_df['MeanRoomsArea'] = melb_df['BuildingArea'] / total_rooms
display(melb_df['MeanRoomsArea'])

Можно ввести ещё один интересный признак — AreaRatio, коэффициент соотношения площади здания (BuildingArea) и площади участка (Landsize). Для этого разницу двух площадей поделим на их сумму:

diff_area = melb_df['BuildingArea'] - melb_df['Landsize']
sum_area = melb_df['BuildingArea'] + melb_df['Landsize']
melb_df['AreaRatio'] = diff_area/sum_area
display(melb_df['AreaRatio'])

Что показывает такой коэффициент? Если присмотреться, можно увидеть, что AreaRatio лежит в интервале от -1 до 1.

Рассмотрим три случая, чтобы понять его значение:

Если рассматриваемые площади равны, то числитель дроби зануляется и коэффициент тоже равен 0.
Если одна из площадей начинает доминировать над другой, то коэффициент начинает расти в отрицательную сторону, если площадь участка больше площади здания, и в положительную сторону, если наоборот.
Наконец, в предельном случае, если площадь здания равна 0, то числитель дроби равен знаменателю со знаком минус, а коэффициент равен -1, а если площадь участка равна 0, то числитель дроби равен знаменателю со знаком плюс, а коэффициент равен 1.
Таким образом, значение в столбце AreaRatio служит своеобразным указателем соотношения площадей объекта недвижимости. Для пустырей — участков без строений — он будет равен -1, для домов без территории — 1, во всех остальных случаях мы можем увидеть, какая площадь больше — здания или участка.


ПРИЗНАКИ ДАТЫ И ВРЕМЕНИ

→ При генерации новых признаков очень ценным может стать временной признак (признак даты и времени). Это особый тип данных, с которым приходится сталкиваться в большинстве задач по обработке данных. В реальных задачах часто нужно сравнивать даты, выделять день недели или час, вычислять различные интервалы между датами. 

Например, в наших данных об объектах недвижимости есть признак даты продажи — столбец Date. Из данного признака можно выделить массу полезной информации, например год, месяц или день недели продажи имущества. На рынке недвижимости, как известно, присутствует сезонность: есть периоды, когда недвижимость покупается чаще, а есть интервалы времени, когда рынок претерпевает застой, поэтому было бы неплохо учитывать эту сезонность при анализе рынка.

ФОРМАТ DATETIME

В жизни мы видим даты в привычных для нас форматах. Например, запись 01.12.18 обычно означает 1 декабря 2018 года. Однако для жителей США эта дата окажется 12 января 2018 года, так как для них привычнее сначала указывать номер месяца, а затем день. 

Многие выгрузки из систем и баз данных имеют свой служебный формат. Например, формат времени из разных систем может отличаться:

2018-11-09 15:45:21;
11/09/2018 3:45:20 PM;
2018-11-09T15:45:21.2984.
Для всех этих случаев необходимо задавать формат распознавания дат и уметь сравнивать их между собой. Для этого был создан единый способ обозначения даты и времени. 

Таким форматом в Pandas является формат datetime, который записывается как YYYY-MM-DD HH: MM: SS, то есть составляющие времени указываются в следующем порядке: год, месяц, день, час, минута, секунда.

В наших данных дата записана в виде DD/MM/YYYY, например 3/12/2017. Посмотрим на это:

display(melb_df['Date'])

Для того чтобы преобразовывать столбцы с датами, записанными в распространённых форматах, в формат datetime, можно воспользоваться функцией pandas.to_datetime(). В нашем случае в функции нужно указать параметр dayfirst=True, который будет обозначать, что в первоначальном признаке первым идет день. Преобразуем столбец Date в формат datetime, передав его в эту функцию:

melb_df['Date'] = pd.to_datetime(melb_df['Date'], dayfirst=True)
display(melb_df['Date'])

В результате мы переопределяем признак Date в формат datetime. При этом так как в изначальном варианте время не было указано, то и после преобразования оно опускается.

Стоит обратить внимание, что изменился тип данных для столбца Date, теперь его тип — datetime64. Рассмотрим несколько возможностей этого типа данных.

ВЫДЕЛЕНИЕ АТРИБУТОВ DATETIME

Тип данных datetime позволяет с помощью специального аксессора dt выделять составляющие времени из каждого элемента столбца, такие как:

date — дата;
year, month, day — год, месяц, день;
time — время;
hour, minute, second — час, минута, секунда;
dayofweek — номер дня недели, от 0 до 6, где 0 — понедельник, 6 — воскресенье;
day_name — название дня недели;
dayofyear — порядковый день года;
quarter — квартал (интервал в три месяца).

Аксессор — это атрибут столбца, хранящий переменные, которые были строковым представлением времени, а затем были изменены с помощью pd.to_datetime().

Обратите внимание, что вы не сможете обратиться к аксессору, если ваш столбец не приведён к типу datetime.

Например, обратившись по атрибуту dt.year в столбце Date, мы можем «достать» год продажи и понять, за какой интервал времени (в годах) представлены наши данные, а также на какой год приходится наибольшее число продаж:

years_sold = melb_df['Date'].dt.year
print(years_sold)
print('Min year sold:', years_sold.min())
print('Max year sold:', years_sold.max())
print('Mode year sold:', years_sold.mode()[0])

В результате обращения к атрибуту datetime melb_df['Date'].dt.year мы получаем объект Series, в котором в качестве значений выступают годы продажи объектов недвижимости. Мы можем занести результат в переменную year_sold и далее работать с ней как с обычным столбцом Series — вычислять максимум, минимум и модальное значение.

Из результатов видно, что данные представлены за интервал с 2016 по 2017 год и наибольшее количество объектов было продано в 2017 году.

Примечание. Так как модальных значений в столбце может быть несколько, метод mode() возвращает объект Series, даже если мода в данных только одна. Чтобы сохранить стилистику вывода информации о годе продажи и выводить только число, а не Series, мы обращаемся к результату работы метода mode() по индексу 0.

Теперь попробуем понять, на какие месяцы приходится пик продаж объектов недвижимости. Для этого выделим атрибут dt.month и на этот раз занесём результат в столбец MonthSale, а затем найдём относительную частоту продаж для каждого месяца от общего количества продаж — для этого используем метод value_counts() с параметром normalize (вывод в долях):

melb_df['MonthSale'] = melb_df['Date'].dt.month
melb_df['MonthSale'].value_counts(normalize=True)

з результатов становится ясно, что наибольшее количество продаж недвижимости приходится на май, июль и сентябрь (пятый, седьмой и девятый месяцы соответственно). Месяцами застоя при этом являются месяцы — октябрь, февраль и январь (десятый, второй и первый месяцы соответственно).

РАБОТА С ИНТЕРВАЛАМИ

Часто бывает такая ситуация, что необходимо вычислять интервалы между двумя временными промежутками. Например, можно вычислить, сколько дней прошло с 1 января 2016 года до момента продажи объекта. Для этого можно просто найти разницу между датами продаж и заявленной датой, представленной в формате datetime:

delta_days = melb_df['Date'] - pd.to_datetime('2016-01-01') 
display(delta_days)

В результате мы получаем Series, элементами которой является количество дней, которое прошло с 1 января 2016 года. Обратите внимание, что данные такого формата относятся к типу timedelta.

Чтобы превратить количество дней из формата интервала в формат целого числа дней, можно воспользоваться аксессором dt для формата timedelta и извлечь из него атрибут days:

display(delta_days.dt.days)

Рассмотрим другой пример. Давайте создадим признак возраста объекта недвижимости в годах на момент продажи. Для этого выделим из столбца с датой продажи год и вычтем из него год постройки здания. Результат оформим в виде столбца AgeBuilding:

melb_df['AgeBuilding'] = melb_df['Date'].dt.year - melb_df['YearBuilt']
display(melb_df['AgeBuilding'])

Примечание. Обратите внимание, что, так как года кодируются целым числом, в результате мы тоже получаем целочисленный столбец — тип int64 (а не timedelta).

На самом деле столбец AgeBuilding дублирует информацию столбца YearBuilt, так как, зная год постройки здания, мы автоматически знаем его возраст. Такие признаки не стоит оставлять вместе, поэтому оставим возраст здания, так как он является более наглядным, а год постройки удалим из таблицы:

melb_df = melb_df.drop('YearBuilt', axis=1)

Для таких случаев Pandas не имеет специальных методов, однако позволяет расширить свою функциональность за счёт использования пользовательских функций. 

Мы можем написать некоторую функцию, которая принимает на вход один элемент столбца, каким-то образом его обрабатывает и возвращает результат, после чего применить эту функцию к каждому элементу в столбце с помощью специального метода apply(). В результате применения этой функции будет возвращён объект Series, элементы которого будут представлять результат работы этой функции.

Рассмотрим пример. В наших данных есть столбец с адресами объектов недвижимости. Проблема этого столбца в том, что в нём слишком большое количество уникальных значений: почти на каждый объект недвижимости в таблице приходится свой уникальный адрес. Убедимся в этом, вычислив количество уникальных значений в столбце с помощью метода nunique():

print(melb_df['Address'].nunique())
# 13378
Если мы прогнозируем цену объекта, то такое большое количество возможных категорий может плохо сказаться на модели, которую мы бы хотели в дальнейшем построить на наших данных. Говорят, что такой признак, скорее всего, не имеет статистической значимости, потому что не позволяет разделить данные на группы, которые можно сравнить по целевому признаку.

Из-за таких признаков зависимость между целевым признаком, который мы хотим предсказать, и признаками, на основе которых мы делаем предсказание, становится очень сложной. При этом точность моделирования при учёте такого признака может не повыситься, а даже снизиться, а производительность однозначно резко упадёт.

Обычно подобные признаки удаляют, однако можно поступить умнее: давайте извлечём из признака адреса характеристику подтипа улицы (улица, шоссе, авеню, бульвар). Для этого сначала внимательнее посмотрим на структуру адреса, выберем несколько строк столбца Address:

print(melb_df['Address'].loc[177])
print(melb_df['Address'].loc[1812])
print(melb_df['Address'].loc[9001])
# 2/119 Railway St N
# 9/400 Dandenong Rd
# 172 Danks St
Итак, адрес строится следующим образом: сначала указывается номер дома и корпус, после указывается название улицы, а в конце — подтип улицы, но в некоторых случаях к подтипу добавляется географическая отметка (N — север, S — юг и т. д.), она нам не нужна . Для того чтобы выделить подтип улицы, на которой находится объект, можно использовать следующую функцию:

# На вход данной функции поступает строка с адресом.
def get_street_type(address):
# Создаём список географических пометок exclude_list.
    exclude_list = ['N', 'S', 'W', 'E']
# Метод split() разбивает строку на слова по пробелу.
# В результате получаем список слов в строке и заносим его в переменную address_list.
    address_list = address.split(' ')
# Обрезаем список, оставляя в нём только последний элемент,
# потенциальный подтип улицы, и заносим в переменную street_type.
    street_type = address_list[-1]
# Делаем проверку на то, что полученный подтип является географической пометкой.
# Для этого проверяем его на наличие в списке exclude_list.
    if street_type in exclude_list:
# Если переменная street_type является географической пометкой,
# переопределяем её на второй элемент с конца списка address_list.
        street_type = address_list[-2]
# Возвращаем переменную street_type, в которой хранится подтип улицы.
    return street_type

Теперь применим эту функцию к столбцу c адресом. Для этого передадим функцию get_street_type в аргумент метода столбца apply(). В результате получим объект Series, который положим в переменную street_types:

street_types = melb_df['Address'].apply(get_street_type)
display(street_types)

Обратите внимание, что функция пишется для одного элемента столбца, а метод apply() применяется к каждому его элементу. Используемая функция обязательно должна иметь возвращаемое значение.

Итак, мы смогли выделить подтип улицы. Посмотрим, сколько уникальных значений у нас получилось:

print(street_types.nunique())
# 56
У нас есть 56 уникальных значений. Однако наш результат можно улучшить. Давайте для начала посмотрим на частоту каждого подтипа улицы с помощью метода value_counts:

display(street_types.value_counts())

Из данного вывода можно увидеть, что есть группа наиболее популярных подтипов улиц, а дальше частота подтипов быстро падает.

В таком случае давайте применим очень распространённый метод уменьшения количества уникальных категорий — выделим n подтипов, которые встречаются чаще всего, а остальные обозначим как 'other' (другие).

Для этого к результату метода value_counts применим метод nlargest(), который возвращает n наибольших значений из Series. Зададим n=10, т. е. мы хотим отобрать десять наиболее популярных подтипов. Извлечём их названия с помощью атрибута index, а результат занесём в переменную popular_stypes:

popular_stypes =street_types.value_counts().nlargest(10).index
print(popular_stypes)
# Index(['St', 'Rd', 'Ct', 'Dr', 'Av', 'Gr', 'Pde', 'Pl', 'Cr', 'Cl'], dtype='object')
Попробуйте применить использованные методы последовательно и выводить результат, чтобы проследить за логикой преобразования.

Теперь, когда у нас есть список наиболее популярных подтипов улиц, введём lambda-функцию, которая будет проверять, есть ли строка x в этом перечне, и, если это так, lambda-функция будет возвращать x, в противном случае она будет возвращать строку 'other'. Наконец, применим такую функцию к Series street_types, полученной ранее, а результат определим в новый столбец таблицы StreetType:

melb_df['StreetType'] = street_types.apply(lambda x: x if x in popular_stypes else 'other')
display(melb_df['StreetType'])

Посмотрим на результирующее число уникальных подтипов:

print(melb_df['StreetType'].nunique())
# 11
Теперь, у нас нет потребности хранить признак Address, так как, если конкретное местоположение объекта всё же и влияет на его стоимость, то оно определяется столбцами Longitude и Lattitude. Удалим его из нашей таблицы:

melb_df = melb_df.drop('Address', axis=1)
Таким образом, с помощью написания собственных функций и их комбинирования с методом apply() из библиотеки Pandas мы смогли извлечь информацию из признака с адресом и заменить на признак подтипа улицы.

Примечание. Внимательный читатель наверняка обратит внимание на то, что мы допустили небольшую ошибку!

Если присмотреться, то в списке подтипов улиц street_types можно заметить подтипы, которые именуются различным образом, но при этом обозначают одинаковые вещи. Например, подтипы Av и Avenue, Bvd и Boulevard, Pde и Parade. Мы упустили данный момент, хотя в реальных задачах стоит обращать пристальное внимание на результаты преобразований и исправлять неточности в данных.

Такие ошибки в данных (обозначение идентичных категорий различными именами) являются одним из видов «грязных» данных.

Порой отследить такие неточности бывает очень сложно, а при наличии большого количества категорий (например, более ста) — практически невозможно.

Мы предлагаем вам самостоятельно разобраться с этой ошибкой: попробуйте написать функцию-преобразование (lambda-функцию-преобразование), которая возвращала бы вместо значений Avenue, Boulevard и Parade их топографические сокращения, и примените её к данным о подтипах улиц.

→ Обратите внимание, что данное преобразование необходимо применить до сокращения количества уникальных категорий.

Резюмируя, поделимся общими рекомендациями по уменьшению числа уникальных значений в признаке, который описывается категориями:

1) Определите (хотя бы на глаз) соотношение числа уникальных категорий интересующего вас признака к общему числу объектов в таблице. Если это соотношение превышает значение 30 %, то это уже повод задуматься над уменьшением числа категорий и перейти к шагу 2.

2) Если ваш признак уникален для каждого объекта, например адрес, имя или название, то такой признак, скорее всего, не имеет статистической значимости. От таких признаков чаще всего избавляются. Однако можно попробовать выделить из этого признака какие-то общие черты, например, как мы это сделали с подтипами улиц. Такой же трюк можно произвести, например, с названиями компаний, в которых может быть скрыт признак типа организации (из строки «ООО Три Слепые Мыши» можно извлечь ООО — общество с ограниченной ответственностью).

3) Если даже после преобразования число уникальных категорий всё ещё велико, можно попробовать с помощью метода value_counts() оценить, есть ли в данных категории, которые употребляются гораздо реже, чем остальные. Если такие категории присутствуют, переходите к шагу 4.

4) Можно подобрать число  популярных категорий таким образом, чтобы эти категории покрывали большую часть ваших данных.

Когда вы выбрали оптимальное число, переходите к шагу 5.

5) Наконец, можно совершить преобразование, обозначив категории, не попавшие в число популярных, как «другие».

→ Такая методика является очень популярной и хорошо показывает себя на практике. Однако не нужно ей злоупотреблять: применяя эту методику ко всем столбцам подряд, вы рискуете потерять немалую долю полезной информации из ваших данных. Внимательно изучите интересующий вас признак, прежде чем преобразовывать его.


ПРИЗНАКИ: КАТЕГОРИАЛЬНЫЕ И ЧИСЛОВЫЕ

Рассмотрим такие статистические термины, как категориальные и числовые признаки.

Под числовыми признаками обычно подразумевают признаки, которые отражают количественную меру и могут принимать значения из неограниченного диапазона.

Числовые признаки могут быть:

дискретными (например, количество комнат, пациентов, дней, отток сотрудников);
непрерывными (например, масса, цена, площадь).
Дискретные признаки чаще всего представлены целыми числами, а непрерывные — целыми числами и числами с плавающей точкой.

Под категориальными признаками обычно подразумевают столбцы в таблице, которые обозначают принадлежность объекта к какому-то классу/категории.

Категориальные признаки могут быть:

номинальными (например, пол, национальность, район);
порядковыми (например, уровень образования, уровень комфорта, стадия заболевания).
Такие признаки имеют ограниченный набор значений. Они чаще всего представлены в виде текстового описания и кодируются в Pandas типом данных object.

Однако это не всегда так. Например, созданный нами ранее признак месяца продажи кодируется числом (от 1 до 12), но на самом деле является категориальным, поскольку диапазон его значений ограничен и каждому числу мы можем поставить в соответствие название месяца.

Возникает вопрос: а какие признаки стоит считать категориальными?

Однозначного ответа на этот вопрос нет. Решение, какой признак отнести к классу категорий, остаётся за исследователем. Некоторые специалисты даже относят количественные признаки в разряд категориальных, когда диапазон возможных значений довольно мал. 

Не существует жёсткого правила, определяющего, сколько значений должна иметь категориальная переменная. Вы должны использовать собственные знания о предметной области, чтобы сделать выбор. Однако в этом разделе мы всё же введём некоторые рекомендации.

ПОЧЕМУ ТАК ВАЖНО ОТЛИЧАТЬ КАТЕГОРИАЛЬНЫЕ СТОЛБЦЫ ОТ ДРУГИХ?

→ Оказывается, анализ и предобработка категориальных признаков отличается от предобработки числовых признаков.

Мы уже видели, что в столбцах с типом данных object мы не можем рассчитать среднее значение, стандартное отклонение или другие статистические параметры, которые годятся только для чисел.

Способ заполнения пропущенных данных и поиск аномальных значений также различаются для разных типов признаков, но этот момент мы рассмотрим в модуле по очистке данных.

КАТЕГОРИИ В ДАННЫХ О НЕДВИЖИМОСТИ

Давайте определим число уникальных категорий в каждом столбце нашей таблицы melb_df. Для этого создадим вспомогательную таблицу unique_counts:

# создаём пустой список
unique_list = []
# пробегаемся по именам столбцов в таблице
for col in melb_df.columns:
    # создаём кортеж (имя столбца, число уникальных значений)
    item = (col, melb_df[col].nunique(),melb_df[col].dtypes) 
    # добавляем кортеж в список
    unique_list.append(item) 
# создаём вспомогательную таблицу и сортируем её
unique_counts = pd.DataFrame(
    unique_list,
    columns=['Column_Name', 'Num_Unique', 'Type']
).sort_values(by='Num_Unique',  ignore_index=True)
# выводим её на экран
display(unique_counts)

Разберём код подробнее:

1) Создаём пустой список, в который будем добавлять кортежи: имя столбца, количество уникальных значений в нём и тип столбца.

2) В цикле перебираем имена столбцов, которые получаем с помощью атрибута columns. В переменной col на каждой итерации находятся имена столбцов — обращаемся к ним в цикле и извлекаем число уникальных элементов с помощью метода nunique(), а также тип столбца с помощью атрибута dtypes. Результат заносим в кортеж и добавляем его в список.

3) Из списка с кортежами (имя столбца, количество уникальных значений в нём, тип столбца) создаём DataFrame, даём названия его столбцам: Column_Name, Num_unique и Type.

4) Сортируем таблицу по столбцу Num_unique в порядке возрастания количества уникальных элементов с помощью метода sort_values() и выводим результат на экран.

Примечание. Мы ещё не изучали сортировку DataFrame методом sort_values() — данную тему мы обсудим в следующем модуле, однако здесь эта функция необходима для более наглядной интерпретации результата.

Итак, с помощью такого несложного кода мы смогли получить вспомогательную таблицу, где в отсортированном виде показано количество уникальных значений во всех столбцах и их типы.

Что интересного мы можем узнать из такой таблицы?

Если присмотреться внимательно, можно увидеть резкий скачок количества уникальных значений, начиная с 14 строки таблицы, где число уникальных значений составляет 152. Учтём этот момент.

Условимся, что категориальными будем считать признаки, у которых число уникальных категорий меньше 150. 

Однако учтём, что признак Date (дата продажи), преобразованный нами ранее в формат datetime, является временным признаком, поэтому далее не будем его воспринимать как категориальный. 

К тому же в наш потенциальный список попали количественные столбцы Rooms, Car, Bedroom и Bathroom. Договоримся, что мы не будем относить их к разряду категориальных, однако, как упоминалось ранее, такое тоже вполне возможно.

Примечание. Ещё раз подчеркиваем, что такая классификация признаков является исключительно субъективной и специфична для задачи.

ТИП ДАННЫХ CATEGORY

Для хранения и оптимизации работы с категориальными признаками в Pandas предусмотрен специальный тип данных — category.

→ Этот тип данных является гибридным: внешне он выглядит как строка, но внутренне представлен массивом целых чисел. Так как данные вместо изначальных строк хранятся в памяти как число, то объём памяти, занимаемой таблицей при использовании типа category, резко уменьшается, что повышает эффективность хранения и работы с таблицей.

→ Более того, этот тип данных расширяет возможности работы с категориальными признаками: мы можем легко преобразовывать категории, строить графики по таким данным (что сложно сделать для типа данных object). Также резко повышается производительность операций, совершаемых с такими столбцами.

Самый простой способ преобразования столбцов к типу данных category — это использование уже знакомого нам метода astype(), в параметры которого достаточно передать строку 'category'.

Рассмотрим это на примере.

Для начала, выведем информацию о памяти, занимаемой текущей таблицей, с помощью метода info():

display(melb_df.info())
Текущий объём памяти — 2.7 Мб.

Примечание. Объём занимаемой памяти и количество столбцов на вашем компьютере может отличаться в зависимости от того, выполняли ли вы код и задания из предыдущих разделов на добавление и преобразование данных в таблице.

Сделаем преобразование столбцов к типу данных category:

cols_to_exclude = ['Date', 'Rooms', 'Bedroom', 'Bathroom', 'Car'] # список столбцов, которые мы не берём во внимание
max_unique_count = 150 # задаём максимальное число уникальных категорий
for col in melb_df.columns: # цикл по именам столбцов
    if melb_df[col].nunique() < max_unique_count and col not in cols_to_exclude: # проверяем условие
        melb_df[col] = melb_df[col].astype('category') # преобразуем тип столбца
display(melb_df.info())

Разберём код подробнее:

1) Задаём список столбцов, которые мы не берём в рассмотрение (cols_to_exclude), а также условленный нами ранее порог уникальных значений столбца max_unique_count.

2) В цикле перебираем имена столбцов, и, если число уникальных категорий меньше заданного порога и имён столбцов нет в списке cols_to_exclude, то с помощью метода astype() приводим столбец к типу данных category.

3) Итоговый объём памяти — 1.9 Мб. В результате такого преобразования объём памяти, занимаемый таблицей, уменьшился почти в 1.5 раза. Это впечатляет!

Особенно хорошо такое преобразование работает на действительно больших данных, где число строк превышает сотни тысяч или миллионы. Иногда изменение типов может уменьшить объём памяти в десятки раз и существенно увеличить производительность.

ПОЛУЧЕНИЕ АТРИБУТОВ CATEGORY

У типа данных category есть свой специальный аксесcор cat, который позволяет получать информацию о своих значениях и преобразовывать их. Например, с помощью атрибута этого аксессора categories мы можем получить список уникальных категорий в столбце Regionname:

print(melb_df['Regionname'].cat.categories)

А теперь посмотрим, каким образом столбец кодируется в виде чисел в памяти компьютера. Для этого можно воспользоваться атрибутом codes:

display(melb_df['Regionname'].cat.codes)

С помощью метода аксессора rename_categories() можно легко переименовать текущие значения категорий. Для этого в данный метод нужно передать словарь, ключи которого — старые имена категорий, а значения — новые.

Рассмотрим на примере: переименуем категории признака типа постройки Type — заменим их на полные названия (напомним, u — unit, h — house, t — townhouse).

melb_df['Type'] = melb_df['Type'].cat.rename_categories({
    'u': 'unit',
    't': 'townhouse',
    'h': 'house'
})
display(melb_df['Type'])

ПОДВОДНЫЕ КАМНИ

А теперь представим ситуацию, что появилась новая партия домов и теперь мы продаём и квартиры (flat). Создадим объект Series new_houses_types, в котором будем хранить типы зданий новой партии домов. Преобразуем тип new_houses_types в такой же тип, как и у столбца Type в таблице melb_data, и выведем результат на экран:

new_houses_types = pd.Series(['unit', 'house', 'flat', 'flat', 'house'])
new_houses_types = new_houses_types.astype(melb_df['Type'].dtype)
display(new_houses_types)

Хммм... С нашими новыми объектами недвижимости произошло нечто странное. По какой-то причине вместо квартир мы получили пустые значения — NaN.

На самом деле причина проста: тип данных category хранит только категории, которые были объявлены при его инициализации. При встрече с новой, неизвестной ранее категорией, этот тип превратит её в пустое значение, так как он просто не знает о существовании этой категории.

Решить эту проблему на самом деле не сложно. Можно добавить категорию flat в столбец Type с помощью метода акссесора cat add_categories(), в который достаточно просто передать имя новой категории:

melb_df['Type'] = melb_df['Type'].cat.add_categories('flat')
new_houses_types = pd.Series(['unit', 'house', 'flat', 'flat', 'house'])
new_houses_types = new_houses_types.astype(melb_df['Type'].dtype)
display(new_houses_types)

Примечание. Добавление новой категории в столбец Type не отразится на самом столбце — текущие категории не изменятся, однако такое преобразование позволит добавлять в таблицу новые данные о домах с новой категорией — flat.

Из данного примера можно сделать вывод, что если набор категорий в столбце жёстко не зафиксирован и может обновляться в процессе работы, то тип category не является подходящим типом данных для этого столбца или необходимо постоянно писать проверки при обновлении таблицы.

Теперь, когда мы рассмотрели основные моменты и нюансы работы с типом данных category, можно сформулировать несколько рекомендаций по его использованию:

1) Необязательно каждый раз преобразовывать категориальные данные в тип данных category. Зачастую это делается исключительно для оптимизации работы с большими данными.

2) Если набор данных занимает значительный процент используемой оперативной памяти, рассмотрите возможность использования типа category.

3) Если у вас очень серьёзные проблемы с производительностью, обратите внимание на использование типа category.

4) Если вы решили использовать тип category, будьте осторожны при добавлении новой информации в вашу таблицу. Убедитесь, что вы собрали всю необходимую информацию, произведите предобработку данных и только после этого используйте преобразование типов.

In [18]:
import pandas as pd

melb_data = pd.read_csv('C:/Users/ilyad/IDE/study/python_11/melb_data_ps.csv', sep=',')
melb_df = melb_data.copy()
melb_df.head()



,index,Suburb,Address,Rooms,Type,Price,Method,SellerG,Date,Distance,...,Car,Landsize,BuildingArea,YearBuilt,CouncilArea,Lattitude,Longtitude,Regionname,Propertycount,Coordinates
0,0,Abbotsford,85 Turner St,2,h,1480000.0,S,Biggin,3/12/2016,2.5,...,1,202.0,126.0,1970,Yarra,-37.7996,144.9984,Northern Metropolitan,4019,"-37.7996, 144.9984"
1,1,Abbotsford,25 Bloomburg St,2,h,1035000.0,S,Biggin,4/02/2016,2.5,...,0,156.0,79.0,1900,Yarra,-37.8079,144.9934,Northern Metropolitan,4019,"-37.8079, 144.9934"
2,2,Abbotsford,5 Charles St,3,h,1465000.0,SP,Biggin,4/03/2017,2.5,...,0,134.0,150.0,1900,Yarra,-37.8093,144.9944,Northern Metropolitan,4019,"-37.8093, 144.9944"
3,3,Abbotsford,40 Federation La,3,h,850000.0,PI,Biggin,4/03/2017,2.5,...,1,94.0,126.0,1970,Yarra,-37.7969,144.9969,Northern Metropolitan,4019,"-37.7969, 144.9969"
4,4,Abbotsford,55a Park St,4,h,1600000.0,VB,Nelson,4/06/2016,2.5,...,2,120.0,142.0,2014,Yarra,-37.8072,144.9941,Northern Metropolitan,4019,"-37.8072, 144.9941"


In [30]:
import pandas as pd

citibike_tripdata = pd.read_csv('C:/Users/ilyad/IDE/study/python_11/citibike-tripdata.csv', sep=',')
data = citibike_tripdata.copy()
data.head()

#ЗАДАНИЕ 6.3
#print(data['start station id'].mode()[0])

#ЗАДАНИЕ 6.4
#print(data['bikeid'].mode()[0])

#ЗАДАНИЕ 6.5
#mode_usertype = data['usertype'].mode()[0]
#count_mode_user = data[data['usertype'] == mode_usertype].shape[0]
#print(round(count_mode_user / data.shape[0], 2))

#ЗАДАНИЕ 6.6
#male_count = data[data['gender'] == 1].shape[0]
#female_count = data[data['gender'] == 0].shape[0]
#print(max([male_count, female_count]))

#ЗАДАНИЕ 6.8
#data.drop(['start station id', 'end station id'], axis=1, inplace=True)
#print(data.shape[1])

#ЗАДАНИЕ 6.9
#data['age'] = 2018 - data['birth year']
#data.drop(['birth year'], axis=1, inplace=True)
#print(data[data['age'] > 60].shape[0])

#ЗАДАНИЕ 6.10
#data['starttime'] = pd.to_datetime(data['starttime'])
#data['stoptime'] = pd.to_datetime(data['stoptime'])
#data['trip duration'] = (data['stoptime'] - data['starttime'])
#print(data.loc[3, 'trip duration'])

#ЗАДАНИЕ 6.11
#weekday = data['starttime'].dt.dayofweek
#data['weekend'] = weekday.apply(lambda x: 1 if x ==5 or x == 6 else 0)
#data['weekend'].sum()

#ЗАДАНИЕ 6.12
def get_time_of_day(time):
    if 0 <= time <= 6:
        return 'night'
    elif 6 < time <= 12:
        return 'morning'
    elif 12 < time <= 18:
        return 'day'
    elif 18 < time <= 23:
        return 'evening'
    else:
        return 'else'
data['time_of_day'] = data['starttime'].dt.hour.apply(get_time_of_day)
a = data[data['time_of_day'] == 'day'].shape[0]
b = data[data['time_of_day'] == 'night'].shape[0]
print(round(a / b))


AttributeError: Can only use .dt accessor with datetimelike values